In [4]:
# imports
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

In [5]:
# keys for Spotify Developer Authorization
credentials = json.load(open('authorization.json'))
client_id = credentials['client_id']
client_secret = credentials['client_secret']

playlist_index = 0

playlists = json.load(open('playlists_like_dislike.json'))
playlist_uri = playlists[playlist_index]['uri']
like = playlists[playlist_index]['like']

In [6]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [9]:
uri = playlist_uri    # the URI is split by ':' to get the username and playlist ID
username = uri.split(':')[2]
playlist_id = uri.split(':')[2]

In [10]:
results = sp.user_playlist(username, playlist_id, 'tracks')

SpotifyOauthError: error: invalid_client, error_description: Invalid client secret